In [1]:
import requests
from selenium import webdriver
from time import sleep
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import string
import spacy
from nltk.corpus import stopwords
from langdetect import detect
import re
from googletrans import Translator
import googletrans
trans = Translator()
from textblob import TextBlob
import nltk
nltk.downloader.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import numpy as np

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/sinatxe/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## Selenium Driver

In [2]:
#driver = webdriver.Chrome(ChromeDriverManager().install(), options = opciones)
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito') 

In [3]:
driver = "./chromedriver.exe"
driver = webdriver.Chrome(driver,options = opciones)

url2 = "http://estersinatxe.blogspot.com/"
driver.get(url2)
driver.implicitly_wait(2)
driver.find_element_by_css_selector("#cookieChoiceDismiss").click()

tit = []
texto = []

for i in range(1,8): 
    try:
        for p in range(1,8):
            tit.append(driver.find_element_by_css_selector(f"#Blog1 > div.blog-posts.hfeed > div:nth-child({p}) > div > div.post-outer > div > h3 > a").text)
            
            texto.append(driver.find_element_by_css_selector(f"#Blog1 > div.blog-posts.hfeed > div:nth-child({p}) > div > div.post-outer > div").text)
        driver.find_element_by_css_selector("#blog-pager-older-link").click()

    except:
        texto.append("no data")
driver.quit()

<ipython-input-3-00f320552e41>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver,options = opciones)
<ipython-input-3-00f320552e41>:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector("#cookieChoiceDismiss").click()
<ipython-input-3-00f320552e41>:15: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  tit.append(driver.find_element_by_css_selector(f"#Blog1 > div.blog-posts.hfeed > div:nth-child({p}) > div > div.post-outer > div > h3 > a").text)
<ipython-input-3-00f320552e41>:17: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  texto.append(driver.find_element_by_css_selector(f"#Blog1 > div.blog-posts.hfeed > div:nth-child({p}) > div > div.post-outer > div").text)
<ipython-input-3-00f320552e41>:18: DeprecationWarning: find_element_

In [4]:
textos = []
for t in texto[:-1]:
    s = t.split("\n")
    ind = s.index("Enviar por correo electrónico")
    firma = s[1:ind-1] #auitamos el título
    if "Ester Sinatxe" in firma:
        inddos = firma.index("Ester Sinatxe")
        text_def = firma[1:inddos]
        textos.append(" ".join(text_def).strip())
    else:
        textos.append(" ".join(firma).strip())


In [6]:
textos[1]

'Números rojos  Subí la apuesta a que te bajaba los pantalones y acabé comprándole las pilas a mi premio de consolación. Ganaste la partida haciéndome perder el tiempo, creo que también perdí las bragas...  Buscando las llaves en el fondo del bolso encontré el ticket de tus promesas con la fecha de devolución caducada.  Mira que yo como pago al contado nunca me falta un beso, pero como tú para sumar uno más uno siempre fuiste corto de cambio, nunca logramos cuadrar las cuentas ni los triángulos amorosos.  Y ahora, todo son números sangrando en mi libreta cuando escribo en este banco del parque, capital de nuestro reino, sin dar crédito alguno a tus palabras.'

In [ ]:
listando = []
for i in range(len(tit)):
    dicc= {}
    dicc["Titulo"] = tit[i]
    dicc["text"] = textos[i]
    listando.append(dicc)

In [ ]:
textos_df = pd.DataFrame(listando)

In [ ]:
textos_df.head(3)

In [ ]:

def english(col):
    try:
    
        trad = trans.translate(col,dest="en")
        return trad.text
    except:
        return col

In [ ]:
textos_df["english"] = textos_df["text"].apply(english)

In [ ]:
textos_df.rename(columns = {"Titulo":"tit","text":"tex","english":"tex_en"},inplace = True)

In [ ]:
textos_df.head()

- instalar python -m spacy download es_core_news_sm
- instalar python -m spacy download en_core_news_sm


In [ ]:
nlp = spacy.load("en_core_web_sm")
nlp_es = spacy.load("es_core_news_sm")

In [ ]:
# trans = Translator()
# esp = "que tengas un buen day"
# en = trans.translate(esp, dest="en")
# print(en.text)

# trans = Translator(service_urls=['translate.googleapis.com'])
# textil = textos_df.text[0]
# textil_en = trans.translate(textil,dest="en")
# print(textil_en.text)

In [ ]:
stop_es = nlp_es.Defaults.stop_words
stop = nlp.Defaults.stop_words

In [ ]:
# tok = nlp_es(textil)
# filtro = []
# for t in tok:
#     if not t.is_stop:
#         lemma = t.lemma_.lower().strip()
#         if re.search('^[a-zA-Z]+$',lemma):
#             filtro.append(trans.translate(lemma,dest="en").text)
# textillo = " ".join(filtro)


In [ ]:
def tokenizer (txt):    
    tokens = nlp(txt)
    filtradas = []
    for token in tokens:
        if not token.is_stop:
            lemma = token.lemma_.lower().strip()
            if re.search('^[a-zA-Z]+$',lemma): # Esto me quita las interrogaciones
                filtradas.append(lemma)
    return " ".join(filtradas)

In [ ]:
textos_df.head(3)

In [ ]:
textos_df["tok_en"] = textos_df.tex_en.apply(tokenizer)

In [ ]:
textos_df.head(3)

In [ ]:
textos_en = pd.DataFrame(textos_df)

In [ ]:
textos_en.head(3)

In [ ]:
#textos_en.to_csv("data/text_en.csv")

In [ ]:
def sentimental(col):
    total = []
    
    blob = TextBlob(col)
    total.append(blob.sentiment[0])
    total.append(blob.sentiment[1])
    
    sia = SentimentIntensityAnalyzer()
    polaridad = sia.polarity_scores(col)
    total.append(polaridad["neg"])
    total.append(polaridad["neu"])
    total.append(polaridad["pos"])
    total.append(polaridad["compound"])
    return total

In [ ]:
textos_en["sentiments"] = textos_en.tok_en.apply(sentimental)

In [ ]:
textos_en.head(3)

In [ ]:
text_sent = textos_en
#text_sent.to_csv("data/text_sent.csv")

In [ ]:

def blobping(col):
    total = []    
    blob = TextBlob(col)
    total.append(blob.sentiment[0])
    return total[0]

text_sent["blobpol"] = text_sent["tok_en"].apply(blobping)


In [ ]:
def blobsubing(col):
    total = []    
    blob = TextBlob(col)
    total.append(blob.sentiment[1])
    return total[0]

text_sent["blobsub"] = text_sent["tok_en"].apply(blobsubing)

In [ ]:
def sianeging(col):
    total = []
    sia = SentimentIntensityAnalyzer()
    polaridad = sia.polarity_scores(col)
    total.append(polaridad["neg"])
    return total[0]

text_sent["sianeg"] = text_sent["tok_en"].apply(sianeging)

In [ ]:
def sianeuing(col):
    total = []
    sia = SentimentIntensityAnalyzer()
    polaridad = sia.polarity_scores(col)
    total.append(polaridad["neu"])
    return total[0]

text_sent["sianeu"] = text_sent["tok_en"].apply(sianeuing)

In [ ]:
def siaposing(col):
    total = []
    sia = SentimentIntensityAnalyzer()
    polaridad = sia.polarity_scores(col)
    total.append(polaridad["pos"])
    return total[0]

text_sent["siapos"] = text_sent["tok_en"].apply(siaposing)

In [ ]:
def siacomping(col):
    total = []
    sia = SentimentIntensityAnalyzer()
    polaridad = sia.polarity_scores(col)
    total.append(polaridad["compound"])
    return total[0]

text_sent["siacomp"] = text_sent["tok_en"].apply(siacomping)

In [ ]:
text_compound = text_sent
#text_compound.to_csv("data/text_compound.csv",index=False)

In [ ]:
text_compound.drop("num_pal",axis=1,inplace=True)

In [ ]:
def cuentastok():
    tot = []
    for t in text_compound["tex"]:
        tot.append(len(t.split(" ")))
    return tot
text_compound["num_pal"] = cuentastok()


In [ ]:
#text_compound.to_csv("data/text_compound.csv",index=False)